In [1]:
from IPython.display import display
import dash
import sys,os
if  not os.path.abspath('./') in sys.path:
    sys.path.append(os.path.abspath('./'))
if  not os.path.abspath('../') in sys.path:
    sys.path.append(os.path.abspath('../'))
sys.path.append(os.path.abspath('../../dashgrid'))
sys.path.append(os.path.abspath('../../dashgrid/dashgrid'))
from dashgrid import dgrid
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output,State
import dash_table
import pandas as pd
import numpy as np
import pdb
import traceback
import importlib
import plotly.graph_objs as go
from plotly.graph_objs.layout import Font,Margin
from plotly.offline import  init_notebook_mode, iplot
init_notebook_mode(connected=True)



# importlib.reload(dgrid)


#### Create styles to be used in grids

In [2]:
borderline = 'none' #'solid'
button_style={
    'line-height': '40px',
    'borderWidth': '1px',
    'borderStyle': borderline,
    'borderRadius': '1px',
    'textAlign': 'center',
    'background-color':'#fffff0',
    'vertical-align':'middle',
}
button_style_no_border={
    'line-height': '40px',
    'textAlign': 'center',
    'background-color':'#fffff0',
    'vertical-align':'middle',
}

border_style={
    'line-height': '40px',
    'border':borderline + ' #000',
    'textAlign': 'center',
    'vertical-align':'middle',
}

table_like = {
    'display':'table',
    'width': '100%'
}

# define h4_like because h4 does not play well with dash_table
h4_like = { 
    'display': 'table-cell',
    'textAlign' : 'center',
    'vertical-align' : 'middle',
    'font-size' : '16px',
    'font-weight': 'bold',
    'width': '100%',
}





#### Create a very simple test grid, with grids inside of grids, and no callbacks

In [3]:
def dash_app_1():
    d1 = dgrid.GridItem(html.Div('test 1',style=button_style))
    d2 = dgrid.GridItem(html.Div('test 2',style=button_style))
    g1 = dgrid.create_grid([d1,d2],additional_grid_properties_dict={'border':'none'})
    d3 = dgrid.GridItem(html.Div('test 3',style=button_style))
    g2 = dgrid.create_grid([g1,d3])
    app = dash.Dash()
    app.layout = html.Div(g2)
    return app


#### Create a method that creates dash_table's

In [4]:

def create_dt_div(dtable_id,df_in=None,
                  columns_to_display=None,
                  editable_columns_in=None,
                  title='Dash Table'):
    
    # create list that 
    editable_columns = [] if editable_columns_in is None else editable_columns_in
    datatable_id = dtable_id
    dt = dash_table.DataTable(
        page_current= 0,
        page_size= 100,
#         page_action='native',
#         sort_action='native',
        filter_action='none', # 'fe',
#             content_style='grow',
        style_cell_conditional=[
            {
                'if': {'row_index': 'odd'},
                'backgroundColor': 'rgb(248, 248, 248)'
            }
        ] + [
            {
                'if': {'column_id': c},
                'textAlign': 'left',
            } for c in ['symbol', 'underlying']
        ],

        style_as_list_view=False,
        style_table={
            'maxHeight':'450px','overflowX': 'scroll','overflowY':'scroll'
#             'height':'15','overflowX': 'scroll','overflowY':'scroll'
        } ,
        editable=True,
        css=[{"selector": "table", "rule": "width: 100%;"}],
        id=datatable_id
    )
    if df_in is None:
        df = pd.DataFrame({'no_data':[]})
    else:
        df = df_in.copy()
        if columns_to_display is not None:
            df = df[columns_to_display]                
    dt.data=df.to_dict("rows")
    dt.columns=[{"name": i, "id": i,'editable': True if i in editable_columns else False} for i in df.columns.values]                    
    print(f'dt.columns {dt.columns}')
    s = h4_like
    child_div = html.Div([html.Div(html.Div(title,style=s),style=table_like),dt])
    return child_div

def file_store_transformer(contents):
    if contents is None or len(contents)<=0 or contents[0] is None:
        d =  None
    else:
        d = dgrid.parse_contents(contents).to_dict('rows')
    return d


#### Create a general method for using plotly to create figures

In [5]:

def plotly_plot(df_in,x_column,plot_title=None,
                y_left_label=None,y_right_label=None,
                bar_plot=True,figsize=(16,10),
                number_of_ticks_display=20,
                yaxis2_cols=None):
    ya2c = [] if yaxis2_cols is None else yaxis2_cols
    ycols = [c for c in df_in.columns.values if c != x_column]
    # create tdvals, which will have x axis labels
    td = list(df_in[x_column]) 
    nt = len(df_in)-1 if number_of_ticks_display > len(df_in) else number_of_ticks_display
    spacing = len(td)//nt
    tdvals = td[::spacing]
    
    # create data for graph
    data = []
    # iterate through all ycols to append to data that gets passed to go.Figure
    for ycol in ycols:
        if bar_plot:
            b = go.Bar(x=td,y=df_in[ycol],name=ycol,yaxis='y' if ycol not in ya2c else 'y2')
        else:
            b = go.Scatter(x=td,y=df_in[ycol],name=ycol,yaxis='y' if ycol not in ya2c else 'y2')
        data.append(b)

    # create a layout
    layout = go.Layout(
        title=plot_title,
        xaxis=dict(
            ticktext=tdvals,
            tickvals=tdvals,
            tickangle=45,
            type='category'),
        yaxis=dict(
            title='y main' if y_left_label is None else y_left_label
        ),
        yaxis2=dict(
            title='y alt' if y_right_label is None else y_right_label,
            overlaying='y',
            side='right'),
        margin=Margin(
            b=100
        )        
    )

    fig = go.Figure(data=data,layout=layout)
    return fig

def make_chart_html(figure_id,df,x_column,**kwdargs):
    f = go.Figure()
    if df is not None and len(df)>0:
        f = plotly_plot(df,x_column,**kwdargs)
    gr = dcc.Graph(
            id=figure_id,
            figure=f,               
            )
    gr_html = html.Div(
        gr,
        className='item1',
        style={'margin-right':'auto' ,'margin-left':'auto' ,
               'height': '98%','width':'98%','border':'thin solid'},
        id = f'{figure_id}_html'
    )
    return gr_html


#### Show examples of plotly_plot and make_chart_html

In [6]:
dfff = pd.DataFrame({'c1':['s1','s2'],'v1':[20,30]})
h1 = plotly_plot(dfff,'c1',plot_title='h1')
iplot(h1)
h2 = make_chart_html('g1',dfff,'c1',plot_title='h2').children.figure
iplot(h2)

#### Create a dash app - WITHOUT GRIDS - that displays an Upload button, a dash_table and a dash Graph

In [7]:
def dash_app_2():
    u1 = dcc.Upload(
                id='u1',
                children=html.Div(['enter a csv file']),
                # Allow multiple files to be uploaded
                multiple=False,
                style=button_style)

    df_init = pd.DataFrame({'symbol':['spy','ibm'],'position':[100,-100]})
    cols=['symbol','position']
    s1 = dcc.Store(id='s1',data=df_init.to_dict('rows'))
    s2 = dcc.Store(id='s2')

    dtable_div = create_dt_div('dtable',df_in=df_init,
                               columns_to_display=cols,
                               editable_columns_in=['position'])
    dt1 = html.Div([dtable_div],id='dt1')
    gr = dgrid.GridGraph('g1', 'graph 1',(dt1.id,'data'),
                        df_x_column='symbol',df_y_columns=['position'],plot_bars=True)
    gr1 = gr.html
    gr1.id = 'gr1'

    u1_lambda = lambda value_list: [None,None] if value_list[0] is None else [file_store_transformer(value_list[0]),None]
    u1_comp = dgrid.ComponentWrapper(u1,
                     input__tuples=[(u1.id,'contents')],
                     output_tuples=[(s1.id,'data'),(s2.id,'data')],
                     callback_input_transformer=u1_lambda)

    def dt1_lambda(value_list): 
        print(f'dt1_lambda value_list: {value_list}')
        ret = [None,None]
        try:
            if value_list[0] is not None:
                dict_df = value_list[0]
                df = pd.DataFrame(dict_df)
                dt_div = create_dt_div('dtable',df_in=df,
                            columns_to_display=cols,
                            editable_columns_in=['position'])
                ret =  [dt_div,dict_df]
        except Exception as e:
            print(str(e))
        print(f'dt1_lambda ret: {ret}')
        if ret[0] is None:
            raise ValueError('dt1_lambda value_list has no data.  Callback return is ignored')
        return ret

    dt1_comp = dgrid.ComponentWrapper(dt1,
                     input__tuples=[(s1.id,'data')],
                     output_tuples=[(dt1.id,'children'),('dtable','data')],
                     callback_input_transformer=dt1_lambda)


    def gr1_lambda(value_list): 
        print(f'gr1_lambda value_list: {value_list}')
        ret = [None]
        try:
            if value_list[0] is not None:
                df = pd.DataFrame(value_list[0])
                ret =  [gr.make_chart_figure(df)]
        except Exception as e:
            print(str(e))
        print(f'gr1_lambda ret: {ret}')
        return ret

    gr1_comp = dgrid.ComponentWrapper(gr1,
                     input__tuples=[('dtable','data')],
                     output_tuples=[(gr1.children.id,'figure')],
                     callback_input_transformer=gr1_lambda)

    app = dash.Dash()
    app.layout = html.Div([u1_comp.html,s1,s2,dt1_comp.html,gr1_comp.html])
    u1_comp.callback(app)
    dt1_comp.callback(app)
    gr1_comp.callback(app)
    return app


#### Create app above, but with Grid wrappers

In [8]:
def dash_app_3():
    app = dash_app_2()    
    apc = app.layout.children
    upl = apc[0]
    stores = [apc[1],apc[2]]
    gr = dgrid.create_grid([apc[3],apc[4]])
    app.layout = html.Div([upl,gr]+stores)
    return app


### Define higher level components that inherit ComponentWrapper

In [9]:
class DivComponent(dgrid.ComponentWrapper):
    def __init__(self,component_id,input_component=None,
                 input_component_property='data',
                 initial_children=None,
                 callback_input_transformer=None,
                 style=None,logger=None):
        s = {} if style is None else style
        init_children = [] if initial_children is None else initial_children
        h1 = html.Div(init_children,id=component_id,style=s)
        h1_lambda = (lambda v:[v]) if callback_input_transformer is None else callback_input_transformer
        input_tuple = None if input_component is None else [(input_component.id,input_component_property)]
        super(DivComponent,self).__init__(
                    h1,input__tuples=input_tuple,
                    output_tuples=[(h1.id,'children')],
                    callback_input_transformer=h1_lambda,
                    style=s,
                    logger=logger)
        

class UploadComponent(dgrid.ComponentWrapper):
    def __init__(self,component_id,text=None,style=None,logger=None):
        t = "Choose a File" if text is None else text
        self.component_id = component_id
        u1 = dcc.Upload(
                    id=component_id,
                    children=html.Div([t]),
                    # Allow multiple files to be uploaded
                    multiple=False,
                    style=button_style if style is None else style)
        
        u1_lambda = lambda value_list: [None] if value_list[0] is None else [file_store_transformer(value_list[0])]
        self.s1 = dcc.Store(id=component_id+"_store")
        super(UploadComponent,self).__init__(
                    u1,input__tuples=[(u1.id,'contents')],
                    output_tuples=[(self.s1.id,'data')],
                    callback_input_transformer=u1_lambda,logger=logger)
        
    @dgrid.ComponentWrapper.html.getter
    def html(self):
        return html.Div([self.div,self.s1])

class UploadFileNameDiv(DivComponent):
    def __init__(self,component_id,upload_component):
        super(UploadFileNameDiv,self).__init__(component_id,upload_component,
                input_component_property='filename',
                style=button_style,
                callback_input_transformer=lambda v: [f'Uploaded File: {v}'])
    
    
class DashTableComponent(dgrid.ComponentWrapper):
    def __init__(self,component_id,df_initial,input_component,title=None,
                 editable_columns=None,style=None,logger=None):
        
        # add component_id and html_id to self
        self.component_id = component_id
        self.html_id = self.component_id+'_html'
        input_tuples = [input_component.output_data_tuple]
        
        # create initial div
        dtable_div = create_dt_div(component_id,df_in=df_initial,
                        columns_to_display=df_initial.columns,
                        editable_columns_in=editable_columns,
                        title='Dash Table' if title is None else title)
        
        outer_div = html.Div([dtable_div],id=self.html_id,
                style={
                'margin-right':'auto' ,'margin-left':'auto' ,
                'height': '98%','width':'98%','border':'thin solid'})
        if style is not None:
            for k in style:
                outer_div.style[k] = style[k]
        
        # define dash_table callback using closure so that you don't refer back to 
        #   class instance during the callback
        def _create_dt_lambda(component_id,cols,editable_cols,logger):
            def _dt_lambda(value_list): 
                logger.debug(f'dt_lambda value_list: {value_list}')
                ret = [None,None]
                try:
                    if value_list[0] is not None:
                        dict_df = value_list[0]
                        df = pd.DataFrame(dict_df)
                        dt_div = create_dt_div(component_id,df_in=df,
                                    columns_to_display=cols,
                                    editable_columns_in=editable_cols)
                        ret =  [dt_div,dict_df]
                except Exception as e:
                    logger.warn(str(e))
                logger.debug(f'dt1_lambda ret: {ret}')
                if ret[0] is None:
                    raise ValueError('dt1_lambda value_list has no data.  Callback return is ignored')
                return ret
            return _dt_lambda
        
        
        # do super, but WITHOUT the callback
        super(DashTableComponent,self).__init__(outer_div,
                     input__tuples=input_tuples,
                     output_tuples=[
                         (outer_div.id,'children'),
                         (self.component_id,'data')],
                     callback_input_transformer=lambda v:[None],logger=logger)

        # set the id of this class instance because the input component 
        #   to the super constructor is NOT the main dcc component
        self.id = component_id
        
        # define callback so that it includes self.logger
        dtlam = _create_dt_lambda(self.component_id,df_initial.columns.values,
                                  editable_columns,self.logger)
        self.callback_input_transformer = dtlam
                
        
class GraphComponent(dgrid.ComponentWrapper):
    def __init__(self,component_id,input_component,x_column,y_columns,
                 plot_bars=True,title=None,
                 editable_columns=None,style=None,logger=None):
        # create title
        t = f"Graph {component_id}" if title is None else title
        
        # get input tuple
        input_tuple = input_component.output_data_tuple

        # add component_id and html_id to self
        self.component_id = component_id
        gr_html = make_chart_html(component_id,None,x_column,plot_title=t)

        # define dash_table callback using closure so that you don't refer back to 
        #   class instance during the callback
        def _create_gr_lambda(component_id,x_column,plot_title,logger):
            def gr_lambda(value_list): 
                logger.debug(f'{component_id} gr_lambda value_list: {value_list}')
                ret = [None]
                try:
                    if value_list[0] is not None:
                        df = pd.DataFrame(value_list[0])
                        fig = plotly_plot(df,x_column,plot_title=plot_title)
                        print(f'gr_lambda {fig}')
                        ret =  [fig]
                except Exception as e:
                    traceback.print_exc()
                    logger.warn(f'gr_lambda ERROR:  {str(e)}')
                logger.debug(f'{component_id} gr_lambda ret: {ret}')
                if ret[0] is None:
                    err_mess = f'{component_id} gr_lambda IGNORING callback. NO ERROR'
                    raise ValueError(err_mess)
                return ret
            return gr_lambda

        # set the outer html id
        if style is not None:
            gr_html.style = style
        # do super, but WITHOUT the callback
        super(GraphComponent,self).__init__(gr_html,
                     input__tuples=[input_tuple],
                     output_tuples=[(self.component_id,'figure')],
                     callback_input_transformer=lambda v:[None])
        
        # define callback so that it includes self.logger
        gr_lam = _create_gr_lambda(self.component_id,x_column,t,self.logger)
        self.callback_input_transformer = gr_lam

        


In [10]:
def dash_app_4():
    u1_comp = UploadComponent('u1',
                    'choose a csv portfolio file with symbol and position columns')
    h1_comp = UploadFileNameDiv('h1',u1_comp)
    df_init = pd.DataFrame({'symbol':['spy','ibm'],'position':[100,-100]})

    dt1_comp = DashTableComponent('dt1',df_init,u1_comp,
                                  editable_columns=['position'],style=border_style)
    gr1_comp = GraphComponent('gr1',dt1_comp,
                              x_column='symbol',y_columns=['position'],
                             title='Risk Graph1',style=border_style)
    dt2_comp = DashTableComponent('dt2',df_init,u1_comp,
                                  editable_columns=['position'],style=border_style)
    gr2_comp = GraphComponent('gr2',dt2_comp,
                              x_column='symbol',y_columns=['position'],
                             title='Risk Graph2',style=border_style)
    div_comp = DivComponent('div1',initial_children='bottom div',style=button_style)
    app = dash.Dash()
    app.layout = html.Div([u1_comp.html,h1_comp.html,
                           dt1_comp.html,gr1_comp.html,
                           dt2_comp.html,gr2_comp.html,
                           div_comp.html])
    [c.callback(app) for c in [u1_comp,h1_comp,
                               dt1_comp,gr1_comp,dt2_comp,gr2_comp]]
    return app




In [11]:
    
def dash_app_5():
    app = dash_app_4()
    apc = app.layout.children
    no_border = {'border': '1px solid #000','grid-gap': '10px 10px',
                'grid-template-columns': '1fr 1fr'}
    grid1 = dgrid.create_grid([apc[0],apc[1]],
                additional_grid_properties_dict=no_border)
    grid2 = dgrid.create_grid([apc[2],apc[3]],num_columns=2,
                additional_grid_properties_dict=no_border)
    grid3 = dgrid.create_grid([apc[4],apc[5]],num_columns=2,
                additional_grid_properties_dict=no_border)
    grid4 = dgrid.create_grid([apc[6]],num_columns=1)
    app.layout = html.Div([grid1,grid2,grid3,grid4])
    return app


### Define methods to fetch data, and do useful risk calculations

In [12]:
def get_history(symbol,dt_beg=None,dt_end=None):
    pass

In [13]:
def dash_app_6():
    pass

In [14]:
DASH_APP_TO_RUN = dash_app_5

In [15]:
app = DASH_APP_TO_RUN()
app.run_server(host='127.0.0.1',port=8500)

dt.columns [{'name': 'position', 'id': 'position', 'editable': True}, {'name': 'symbol', 'id': 'symbol', 'editable': False}]
dt.columns [{'name': 'position', 'id': 'position', 'editable': True}, {'name': 'symbol', 'id': 'symbol', 'editable': False}]
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


2019-07-22 12:58:50,508 - werkzeug - INFO -  * Running on http://127.0.0.1:8500/ (Press CTRL+C to quit)
2019-07-22 12:58:56,134 - werkzeug - INFO - 127.0.0.1 - - [22/Jul/2019 12:58:56] "GET / HTTP/1.1" 200 -
2019-07-22 12:58:56,187 - werkzeug - INFO - 127.0.0.1 - - [22/Jul/2019 12:58:56] "GET /_dash-component-suites/dash_renderer/react@16.8.6.min.js?v=1.0.0&m=1563025507 HTTP/1.1" 200 -
2019-07-22 12:58:56,197 - werkzeug - INFO - 127.0.0.1 - - [22/Jul/2019 12:58:56] "GET /_dash-component-suites/dash_renderer/prop-types@15.7.2.min.js?v=1.0.0&m=1563025507 HTTP/1.1" 200 -
2019-07-22 12:58:56,198 - werkzeug - INFO - 127.0.0.1 - - [22/Jul/2019 12:58:56] "GET /_dash-component-suites/dash_renderer/react-dom@16.8.6.min.js?v=1.0.0&m=1563025507 HTTP/1.1" 200 -
2019-07-22 12:58:56,218 - werkzeug - INFO - 127.0.0.1 - - [22/Jul/2019 12:58:56] "GET /_dash-component-suites/dash_html_components/dash_html_components.min.js?v=1.0.0&m=1563025508 HTTP/1.1" 200 -
2019-07-22 12:58:56,219 - werkzeug - INFO - 

2019-07-22 12:58:57,196 - flask.app - ERROR - Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/Users/bperlman1/Virtualenvs3/dashrisk2/lib/python3.6/site-packages/flask/app.py", line 2292, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/bperlman1/Virtualenvs3/dashrisk2/lib/python3.6/site-packages/flask/app.py", line 1815, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/bperlman1/Virtualenvs3/dashrisk2/lib/python3.6/site-packages/flask/app.py", line 1718, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/Users/bperlman1/Virtualenvs3/dashrisk2/lib/python3.6/site-packages/flask/_compat.py", line 35, in reraise
    raise value
  File "/Users/bperlman1/Virtualenvs3/dashrisk2/lib/python3.6/site-packages/flask/app.py", line 1813, in full_dispatch_request
    rv = self.dispatch_request()
  File "/Users/bperlman1/Virtualenvs3/dashrisk2/lib/python3.6/site-packages/flask/app.py", line 

gr_lambda Figure({
    'data': [{'name': 'position',
              'type': 'bar',
              'uid': 'fe6ad578-aca1-11e9-be8c-8c859025e7f9',
              'x': [spy, ibm],
              'y': array([ 100, -100]),
              'yaxis': 'y'}],
    'layout': {'margin': {'b': 100},
               'title': 'Risk Graph1',
               'xaxis': {'tickangle': 45, 'ticktext': ['spy'], 'tickvals': ['spy'], 'type': 'category'},
               'yaxis': {'title': 'y main'},
               'yaxis2': {'overlaying': 'y', 'side': 'right', 'title': 'y alt'}}
})
gr_lambda Figure({
    'data': [{'name': 'position',
              'type': 'bar',
              'uid': 'fe78aa90-aca1-11e9-af63-8c859025e7f9',
              'x': [spy, ibm],
              'y': array([ 100, -100]),
              'yaxis': 'y'}],
    'layout': {'margin': {'b': 100},
               'title': 'Risk Graph2',
               'xaxis': {'tickangle': 45, 'ticktext': ['spy'], 'tickvals': ['spy'], 'type': 'category'},
               'yaxis

2019-07-22 12:59:05,961 - werkzeug - INFO - 127.0.0.1 - - [22/Jul/2019 12:59:05] "POST /_dash-update-component HTTP/1.1" 200 -
2019-07-22 12:59:05,973 - werkzeug - INFO - 127.0.0.1 - - [22/Jul/2019 12:59:05] "POST /_dash-update-component HTTP/1.1" 200 -
2019-07-22 12:59:06,014 - werkzeug - INFO - 127.0.0.1 - - [22/Jul/2019 12:59:06] "POST /_dash-update-component HTTP/1.1" 200 -
2019-07-22 12:59:06,024 - werkzeug - INFO - 127.0.0.1 - - [22/Jul/2019 12:59:06] "POST /_dash-update-component HTTP/1.1" 200 -


dt.columns [{'name': 'position', 'id': 'position', 'editable': True}, {'name': 'symbol', 'id': 'symbol', 'editable': False}]
dt.columns [{'name': 'position', 'id': 'position', 'editable': True}, {'name': 'symbol', 'id': 'symbol', 'editable': False}]
gr_lambda Figure({
    'data': [{'name': 'position',
              'type': 'bar',
              'uid': '03de5c34-aca2-11e9-95f7-8c859025e7f9',
              'x': [ZSY00, ^EURUSD, XLU, XLE, USO, SPY, SOYB, ZCY00, CLY00, HOY00,
                    HGU19, CPER, IBM, AAPL, GOOG, MSFT, ZSY00, ^EURUSD, XLU, XLE,
                    USO, SPY, SOYB, ZCY00, CLY00, HOY00, HGU19, CPER, IBM, AAPL,
                    GOOG, MSFT],
              'y': array([   -50, 200000,   -200,    200,      2,   -200,   3500,    -50,    500,
                           21000,  25000,  -2600,    100,     70,    100,    100,    -50, 200000,
                            -200,    200,      2,   -200,   3500,    -50,    500,  21000,  25000,
                           -2600, 

2019-07-22 12:59:06,244 - werkzeug - INFO - 127.0.0.1 - - [22/Jul/2019 12:59:06] "POST /_dash-update-component HTTP/1.1" 200 -
2019-07-22 12:59:06,252 - werkzeug - INFO - 127.0.0.1 - - [22/Jul/2019 12:59:06] "POST /_dash-update-component HTTP/1.1" 200 -
2019-07-22 12:59:06,257 - werkzeug - INFO - 127.0.0.1 - - [22/Jul/2019 12:59:06] "POST /_dash-update-component HTTP/1.1" 200 -
2019-07-22 12:59:06,279 - werkzeug - INFO - 127.0.0.1 - - [22/Jul/2019 12:59:06] "POST /_dash-update-component HTTP/1.1" 200 -


gr_lambda Figure({
    'data': [{'name': 'position',
              'type': 'bar',
              'uid': '03e70bfa-aca2-11e9-a2b3-8c859025e7f9',
              'x': [ZSY00, ^EURUSD, XLU, XLE, USO, SPY, SOYB, ZCY00, CLY00, HOY00,
                    HGU19, CPER, IBM, AAPL, GOOG, MSFT, ZSY00, ^EURUSD, XLU, XLE,
                    USO, SPY, SOYB, ZCY00, CLY00, HOY00, HGU19, CPER, IBM, AAPL,
                    GOOG, MSFT],
              'y': array([   -50, 200000,   -200,    200,      2,   -200,   3500,    -50,    500,
                           21000,  25000,  -2600,    100,     70,    100,    100,    -50, 200000,
                            -200,    200,      2,   -200,   3500,    -50,    500,  21000,  25000,
                           -2600,    100,     70,    100,    100]),
              'yaxis': 'y'}],
    'layout': {'margin': {'b': 100},
               'title': 'Risk Graph2',
               'xaxis': {'tickangle': 45,
                         'ticktext': [ZSY00, ^EURUSD, XLU, XLE, USO,

In [16]:
def flatten_layout(app,silent=True):
    # define recursive search
    def _recursive_div_display(c,div_list):
    #     pdb.set_trace()
        if  hasattr(c,'children') and 'div.div' in str(type(c)).lower() and len(c.children)>0:
            for c2 in c.children:
                _recursive_div_display(c2,div_list)
        else:
            div_list.append(c)
    
    # run recursive search
    final_list = []
    for c in list(np.array(app.layout.children).reshape(-1)):    
        dlist = []
        _recursive_div_display(c,dlist)
        if not silent:
            print(f'******************** list len = {len(dlist)} *********************')
            for i in range(len(dlist)):
                display(dlist[i])
        final_list.extend(dlist)
    
    # return results
    return final_list
    

In [17]:
l = flatten_layout(app)
for c in l:
    print('***********************************************')
    print(c)

***********************************************
Upload(children=Div(['choose a csv portfolio file with symbol and position columns']), id='u1', multiple=False, style={'line-height': '40px', 'borderWidth': '1px', 'borderStyle': 'none', 'borderRadius': '1px', 'textAlign': 'center', 'background-color': '#fffff0', 'vertical-align': 'middle'})
***********************************************
Store(id='u1_store')
***********************************************
Div(children=[], id='h1', style={'line-height': '40px', 'borderWidth': '1px', 'borderStyle': 'none', 'borderRadius': '1px', 'textAlign': 'center', 'background-color': '#fffff0', 'vertical-align': 'middle'})
***********************************************
DataTable(columns=[{'name': 'position', 'id': 'position', 'editable': True}, {'name': 'symbol', 'id': 'symbol', 'editable': False}], css=[{'selector': 'table', 'rule': 'width: 100%;'}], data=[{'position': 100, 'symbol': 'spy'}, {'position': -100, 'symbol': 'ibm'}], editable=True, id='dt

In [18]:
#!jupyter nbconvert utilities.ipynb --to python